In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score
import numpy as np
import joblib
from tensorflow.keras.preprocessing import image

C:\Users\mf\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
train_dir = 'C:/Users/DELL/Downloads/New_folder/SEC_Five/dataset/train' 
test_dir = 'C:/Users/DELL/Downloads/New_folder/SEC_Five/dataset/test'

In [4]:
# Set the input shape
input_shape = (128, 128, 3)
batch_size = 32

In [5]:
# Use ImageDataGenerator for data augmentation
data_generator = ImageDataGenerator(rescale=1./255)

In [6]:
# Load the training images
train_generator = data_generator.flow_from_directory(
    train_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='binary',  # Binary classification for male and female
    shuffle=False)

Found 12017 images belonging to 2 classes.


In [7]:
# Load the test images
test_generator = data_generator.flow_from_directory(
    test_dir,
    target_size=(input_shape[0], input_shape[1]),
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False)

Found 216 images belonging to 2 classes.


In [8]:
# Define the CNN feature extraction model
model = Sequential([
    Conv2D(32, (3, 3), input_shape=input_shape),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3)),
    Activation('relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128),
    Activation('relu'),
    Dropout(0.5)
])

C:\Users\mf\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [9]:
# Extract features from training images 
train_features = model.predict(train_generator)
train_features = train_features.reshape(train_features.shape[0], -1)  # Flatten features for classifier
train_labels = train_generator.classes

C:\Users\mf\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:120: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


376/376 ━━━━━━━━━━━━━━━━━━━━ 16s 43ms/step


In [10]:
# Extract features from test images 
test_features = model.predict(test_generator)
test_features = test_features.reshape(test_features.shape[0], -1)
test_labels = test_generator.classes

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


In [11]:
# Train CatBoost classifier with optimized parameters
cb_classifier = CatBoostClassifier(iterations=1500, learning_rate=0.05, depth=8, verbose=0)
cb_classifier.fit(train_features, train_labels)

In [12]:
# Make predictions on the test data
test_predictions = cb_classifier.predict(test_features)
# Calculate accuracy
test_accuracy = accuracy_score(test_labels, test_predictions)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 83.33%


In [13]:
# Save the Keras CNN model as an .h5 file
model.save('cnn_feature_extractor_gender.h5')

# Save the trained CatBoost model as a .pkl file
joblib.dump(cb_classifier, 'catboost_classifier_gender.pkl')

['catboost_classifier_gender.pkl']

In [15]:
from tensorflow.keras.models import Sequential, load_model
# Load the saved models
cnn_model = load_model('cnn_feature_extractor_gender.h5')
catboost_classifier = joblib.load('catboost_classifier_gender.pkl')

In [16]:
def classify_image(img_path):
    # Load and preprocess the new image
    img = image.load_img(img_path, target_size=(128, 128))  # Resize to match the CNN input
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    img_array /= 255.0  # Normalize to match the training preprocessing

    # Extract features using the CNN model
    features = cnn_model.predict(img_array)
    features = features.reshape(1, -1)  # Flatten to 1D for CatBoost

    # Classify the features with the CatBoost model
    prediction = catboost_classifier.predict(features)
    
    # Interpret the prediction
    class_labels = {0: 'Male', 1: 'Female'}
    result = class_labels[int(prediction[0])]
    return result

# Test the function with a new image
img_path =  r'C:/Users/DELL/Downloads/New_folder/SEC_Five/dataset/test/male/063848.jpg'
result = classify_image(img_path)
print(f"The image is classified as: {result}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step
The image is classified as: Male
